### Word Group In Youtube

In [229]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [230]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [231]:
# language pair
lang_folder = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 40000  # native word end index
unique_word_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 5  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 45  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [232]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [233]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [234]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [235]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [236]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [237]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [238]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,e,17674355
1,non,14565060
2,che,14355207
3,di,13837236
4,un,9088369
...,...,...
290541,disonorevolmente,5
290542,disonoreresti,5
290543,saffici,5
290544,benenoi,5


In [239]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [240]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,e,17674355
1,non,14565060
2,che,14355207
3,di,13837236
4,un,9088369
...,...,...
39995,guaritori,351
39996,lives,351
39997,facetime,351
39998,wheatus,351


In [241]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [242]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [243]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

39993

#### Youtube Sentence Analysis

In [244]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:01.850,00:00:05.000,siamo gente strana noi italiani,_tTy_SQeOJ0
1,00:00:05.200,00:00:08.120,innamorati delle contraddizioni,_tTy_SQeOJ0
2,00:00:08.420,00:00:11.140,viviamo nel paese più bello del mondo,_tTy_SQeOJ0
3,00:00:11.240,00:00:13.160,e facciamo finta di non saperlo,_tTy_SQeOJ0
4,00:00:14.100,00:00:16.040,e adesso che il paese è vuoto,_tTy_SQeOJ0
...,...,...,...,...
1892051,01:08:00.880,01:08:12.850,potete dire due parole no allora è stato un in...,LNDH1NCiU2w
1892052,01:08:12.850,01:08:20.560,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w
1892053,01:08:20.560,01:08:26.770,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w
1892054,01:08:26.770,01:08:39.700,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w


In [245]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [246]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,1.85,5.00,siamo gente strana noi italiani,_tTy_SQeOJ0
1,1,5.20,8.12,innamorati delle contraddizioni,_tTy_SQeOJ0
2,2,8.42,11.14,viviamo nel paese più bello del mondo,_tTy_SQeOJ0
3,3,11.24,13.16,e facciamo finta di non saperlo,_tTy_SQeOJ0
4,4,14.10,16.04,e adesso che il paese è vuoto,_tTy_SQeOJ0
...,...,...,...,...,...
1892051,1892051,4080.88,4092.85,potete dire due parole no allora è stato un in...,LNDH1NCiU2w
1892052,1892052,4092.85,4100.56,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w
1892053,1892053,4100.56,4106.77,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w
1892054,1892054,4106.77,4119.70,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w


In [247]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [248]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [249]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,1.85,5.00,siamo gente strana noi italiani,_tTy_SQeOJ0,100.0,,"strana, italiani, noi, siamo, gente"
1,1,5.20,8.12,innamorati delle contraddizioni,_tTy_SQeOJ0,100.0,,"innamorati, contraddizioni, delle"
2,2,8.42,11.14,viviamo nel paese più bello del mondo,_tTy_SQeOJ0,100.0,,"paese, bello, mondo, viviamo, del, più, nel"
3,3,11.24,13.16,e facciamo finta di non saperlo,_tTy_SQeOJ0,100.0,,"saperlo, e, finta, non, facciamo, di"
4,4,14.10,16.04,e adesso che il paese è vuoto,_tTy_SQeOJ0,100.0,,"paese, che, adesso, il, e, vuoto, è"
...,...,...,...,...,...,...,...,...
1892051,1892051,4080.88,4092.85,potete dire due parole no allora è stato un in...,LNDH1NCiU2w,92.9,,"stato, allora, dire, due, bello, produttivo, p..."
1892052,1892052,4092.85,4100.56,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w,100.0,,"disponibile, adesso, dover, per, firme, render..."
1892053,1892053,4100.56,4106.77,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w,100.0,,"ancora, che, protagonisti, due, veramente, que..."
1892054,1892054,4106.77,4119.70,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w,91.7,aquilante,"stesse, che, rimbalzo, grazie, favore, firma, ..."


In [250]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,1.85,5.00,siamo gente strana noi italiani,_tTy_SQeOJ0,100.0,NaN,"strana, italiani, noi, siamo, gente"
1,1,5.20,8.12,innamorati delle contraddizioni,_tTy_SQeOJ0,100.0,NaN,"innamorati, contraddizioni, delle"
2,2,8.42,11.14,viviamo nel paese più bello del mondo,_tTy_SQeOJ0,100.0,NaN,"paese, bello, mondo, viviamo, del, più, nel"
3,3,11.24,13.16,e facciamo finta di non saperlo,_tTy_SQeOJ0,100.0,NaN,"saperlo, e, finta, non, facciamo, di"
4,4,14.10,16.04,e adesso che il paese è vuoto,_tTy_SQeOJ0,100.0,NaN,"paese, che, adesso, il, e, vuoto, è"
...,...,...,...,...,...,...,...,...
1892051,1892051,4080.88,4092.85,potete dire due parole no allora è stato un in...,LNDH1NCiU2w,92.9,NaN,"stato, allora, dire, due, bello, produttivo, p..."
1892052,1892052,4092.85,4100.56,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w,100.0,NaN,"disponibile, adesso, dover, per, firme, render..."
1892053,1892053,4100.56,4106.77,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w,100.0,NaN,"ancora, che, protagonisti, due, veramente, que..."
1892054,1892054,4106.77,4119.70,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w,91.7,aquilante,"stesse, che, rimbalzo, grazie, favore, firma, ..."


In [251]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,1.85,5.00,siamo gente strana noi italiani,_tTy_SQeOJ0,100.0,NaN,"strana, italiani, noi, siamo, gente"
1,1,5.20,8.12,innamorati delle contraddizioni,_tTy_SQeOJ0,100.0,NaN,"innamorati, contraddizioni, delle"
2,2,8.42,11.14,viviamo nel paese più bello del mondo,_tTy_SQeOJ0,100.0,NaN,"paese, bello, mondo, viviamo, del, più, nel"
3,3,11.24,13.16,e facciamo finta di non saperlo,_tTy_SQeOJ0,100.0,NaN,"saperlo, e, finta, non, facciamo, di"
4,4,14.10,16.04,e adesso che il paese è vuoto,_tTy_SQeOJ0,100.0,NaN,"paese, che, adesso, il, e, vuoto, è"
...,...,...,...,...,...,...,...,...
1878941,1892051,4080.88,4092.85,potete dire due parole no allora è stato un in...,LNDH1NCiU2w,92.9,NaN,"stato, allora, dire, due, bello, produttivo, p..."
1878942,1892052,4092.85,4100.56,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w,100.0,NaN,"disponibile, adesso, dover, per, firme, render..."
1878943,1892053,4100.56,4106.77,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w,100.0,NaN,"ancora, che, protagonisti, due, veramente, que..."
1878944,1892054,4106.77,4119.70,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w,91.7,aquilante,"stesse, che, rimbalzo, grazie, favore, firma, ..."


In [252]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,0,1.85,5.00,siamo gente strana noi italiani,_tTy_SQeOJ0,100.0,NaN,"strana, italiani, noi, siamo, gente",siamo gente strana noi italiani
2,2,8.42,11.14,viviamo nel paese più bello del mondo,_tTy_SQeOJ0,100.0,NaN,"paese, bello, mondo, viviamo, del, più, nel",viviamo nel paese più bello del mondo
3,3,11.24,13.16,e facciamo finta di non saperlo,_tTy_SQeOJ0,100.0,NaN,"saperlo, e, finta, non, facciamo, di",e facciamo finta di non saperlo
4,4,14.10,16.04,e adesso che il paese è vuoto,_tTy_SQeOJ0,100.0,NaN,"paese, che, adesso, il, e, vuoto, è",e adesso che il paese è vuoto
5,5,16.54,18.94,e lo possiamo ammirare solo dalla finestra,_tTy_SQeOJ0,100.0,NaN,"dalla, ammirare, solo, e, finestra, possiamo, lo",e lo possiamo ammirare solo dalla finestra
...,...,...,...,...,...,...,...,...,...
1878941,1892051,4080.88,4092.85,potete dire due parole no allora è stato un in...,LNDH1NCiU2w,92.9,NaN,"stato, allora, dire, due, bello, produttivo, p...",potete dire due parole no allora è stato un in...
1878942,1892052,4092.85,4100.56,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w,100.0,NaN,"disponibile, adesso, dover, per, firme, render...",molto curioso quindi adesso dover rendere disp...
1878943,1892053,4100.56,4106.77,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w,100.0,NaN,"ancora, che, protagonisti, due, veramente, que...",volesse fare ringraziamo ancora questo incontr...
1878944,1892054,4106.77,4119.70,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w,91.7,aquilante,"stesse, che, rimbalzo, grazie, favore, firma, ...",strano rimbalzo grazie stesse materie del file


In [253]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,siamo gente strana noi italiani,1.85,5.000000,siamo gente strana noi italiani,_tTy_SQeOJ0
1,viviamo nel paese più bello del mondo,8.42,11.140000,viviamo nel paese più bello del mondo,_tTy_SQeOJ0
2,e facciamo finta di non saperlo,11.24,13.160000,e facciamo finta di non saperlo,_tTy_SQeOJ0
3,e adesso che il paese è vuoto,14.10,16.040000,e adesso che il paese è vuoto,_tTy_SQeOJ0
4,e lo possiamo ammirare solo dalla finestra,16.54,18.940000,e lo possiamo ammirare solo dalla finestra,_tTy_SQeOJ0
...,...,...,...,...,...
1452544,potete dire due parole no allora è stato un in...,4080.88,4092.850000,potete dire due parole no allora è stato un in...,LNDH1NCiU2w
1452545,molto curioso quindi adesso dover rendere disp...,4092.85,4100.560000,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w
1452546,volesse fare ringraziamo ancora questo incontr...,4100.56,4106.770000,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w
1452547,strano rimbalzo grazie stesse materie del file,4106.77,4114.662338,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w


In [254]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,siamo gente strana noi italiani,1.55,5.300000,siamo gente strana noi italiani,_tTy_SQeOJ0
1,viviamo nel paese più bello del mondo,8.12,11.440000,viviamo nel paese più bello del mondo,_tTy_SQeOJ0
2,e facciamo finta di non saperlo,10.94,13.460000,e facciamo finta di non saperlo,_tTy_SQeOJ0
3,e adesso che il paese è vuoto,13.80,16.340000,e adesso che il paese è vuoto,_tTy_SQeOJ0
4,e lo possiamo ammirare solo dalla finestra,16.24,19.240000,e lo possiamo ammirare solo dalla finestra,_tTy_SQeOJ0
...,...,...,...,...,...
1452544,potete dire due parole no allora è stato un in...,4080.58,4093.150000,potete dire due parole no allora è stato un in...,LNDH1NCiU2w
1452545,molto curioso quindi adesso dover rendere disp...,4092.55,4100.860000,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w
1452546,volesse fare ringraziamo ancora questo incontr...,4100.26,4107.070000,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w
1452547,strano rimbalzo grazie stesse materie del file,4106.47,4114.962338,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w


In [255]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,siamo gente strana noi italiani,2,5,siamo gente strana noi italiani,_tTy_SQeOJ0
1,viviamo nel paese più bello del mondo,8,11,viviamo nel paese più bello del mondo,_tTy_SQeOJ0
2,e facciamo finta di non saperlo,11,13,e facciamo finta di non saperlo,_tTy_SQeOJ0
3,e adesso che il paese è vuoto,14,16,e adesso che il paese è vuoto,_tTy_SQeOJ0
4,e lo possiamo ammirare solo dalla finestra,16,19,e lo possiamo ammirare solo dalla finestra,_tTy_SQeOJ0
...,...,...,...,...,...
1452544,potete dire due parole no allora è stato un in...,4081,4093,potete dire due parole no allora è stato un in...,LNDH1NCiU2w
1452545,molto curioso quindi adesso dover rendere disp...,4093,4101,molto curioso quindi adesso dover rendere disp...,LNDH1NCiU2w
1452546,volesse fare ringraziamo ancora questo incontr...,4100,4107,volesse fare ringraziamo ancora questo incontr...,LNDH1NCiU2w
1452547,strano rimbalzo grazie stesse materie del file,4106,4115,strano rimbalzo grazie stesse materie del file...,LNDH1NCiU2w


In [256]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,perchè possiamo vederlo con le mani cioè per f...,578,630,perchè possiamo vederlo con le mani cioè per f...,dMNIjzeZA1c
1,magari tutte le cose i luoghi della cultura de...,1000,1043,si parla di adeguare magari tutte le cose i lu...,dMNIjzeZA1c
2,musica rilassante per studiare musica rilassan...,0,2,musica rilassante per studiare musica rilassan...,vH8zDccAnxM
3,a cui avanzano i soldi io non lho mai vista al...,911,944,avete mai visto unamministrazione a cui avanza...,dMNIjzeZA1c
4,spesso e volentieri quando ero più piccola la ...,702,722,spesso e volentieri quando ero più piccola la ...,uB0AorgbwTc
...,...,...,...,...,...
1452544,c p t f q,429,433,se è preceduta da una consonante se è seguita ...,LuqYN6dv31g
1452545,j k w x y,29,35,j k w x y,HA4ahOkkcn8
1452546,l m n o p,41,43,l m n o p,1QkhR6KbQm4
1452547,m z y p w,3671,3672,lvzl nnv m z y p w,RCQ0CDuogVM


In [257]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,perchè possiamo vederlo con le mani cioè per f...,578,630,perchè possiamo vederlo con le mani cioè per f...,dMNIjzeZA1c,79
1,magari tutte le cose i luoghi della cultura de...,1000,1043,si parla di adeguare magari tutte le cose i lu...,dMNIjzeZA1c,71
2,musica rilassante per studiare musica rilassan...,0,2,musica rilassante per studiare musica rilassan...,vH8zDccAnxM,38
3,a cui avanzano i soldi io non lho mai vista al...,911,944,avete mai visto unamministrazione a cui avanza...,dMNIjzeZA1c,71
4,spesso e volentieri quando ero più piccola la ...,702,722,spesso e volentieri quando ero più piccola la ...,uB0AorgbwTc,85
...,...,...,...,...,...,...
1452544,c p t f q,429,433,se è preceduta da una consonante se è seguita ...,LuqYN6dv31g,100
1452545,j k w x y,29,35,j k w x y,HA4ahOkkcn8,100
1452546,l m n o p,41,43,l m n o p,1QkhR6KbQm4,100
1452547,m z y p w,3671,3672,lvzl nnv m z y p w,RCQ0CDuogVM,100


In [258]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk
1,il linguaggio il racconto dello straordinario ...,704,727,e un lavoro nel senso che nella direzione dall...,Ko4c8DMTlN0
2,quindi questo dizionario ha portato infinità d...,758,790,quindi questo dizionario ha portato infinità d...,8ncnMXgDuIU
3,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A
4,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c
...,...,...,...,...,...
1148601,l m n o p,22,24,l m n o p,1QkhR6KbQm4
1148602,c p t f q,429,433,se è preceduta da una consonante se è seguita ...,LuqYN6dv31g
1148603,j k w x y,29,35,j k w x y,HA4ahOkkcn8
1148604,l m n o p,41,43,l m n o p,1QkhR6KbQm4


In [259]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-259-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,100
1,il linguaggio il racconto dello straordinario ...,704,727,e un lavoro nel senso che nella direzione dall...,Ko4c8DMTlN0,38
2,quindi questo dizionario ha portato infinità d...,758,790,quindi questo dizionario ha portato infinità d...,8ncnMXgDuIU,60
3,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,95
4,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,100
...,...,...,...,...,...,...
1148601,l m n o p,22,24,l m n o p,1QkhR6KbQm4,100
1148602,c p t f q,429,433,se è preceduta da una consonante se è seguita ...,LuqYN6dv31g,12
1148603,j k w x y,29,35,j k w x y,HA4ahOkkcn8,100
1148604,l m n o p,41,43,l m n o p,1QkhR6KbQm4,100


In [260]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s
...,...,...,...,...,...
574804,e due di b a,998,1006,e due di b a,DzKXR5ru1rg
574805,a e o i u,10,13,a e o i u,QO_6yCw0l8M
574806,l m n o p,22,24,l m n o p,1QkhR6KbQm4
574807,j k w x y,29,35,j k w x y,HA4ahOkkcn8


In [261]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,altre persone pensano se lui spalla così dei s...,zzThbaowBPo
1,ciascuno dalla propria epoca temporale,zyN6zW1wl3A
2,porta risultati oggi primo posto,zxRUSBG7ShA
3,sai sentiti dire cioè noi vogliamo solo portar...,zw9tEaHmzxs
4,questa ricetta è tradizionalmente a parte ques...,zuig141MolU
...,...,...
4116,qualche volta dovremo farlo insieme,04p43O_EbqY
4117,informazioni su come poter avere questi conten...,04DA8gYIuwE
4118,uno due tre quattro cinque sei sette otto allo...,03b5JepSn3c
4119,scomparsa purtroppo dai nostri occhi ormai dal,012TO0CVOfE


In [262]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s
...,...,...,...,...,...
4101,can we do it again,243,244,can we do it again,GTQHOUT5T5E
4102,sì va be my money,2154,2166,sì va be my money,j0ZS0wJ5XDk
4103,so what did he do,103,106,so what did he do,1wxyoR21bo0
4104,we have y and t,31,34,we have y and t,h1Xh1JJuh_I


In [263]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,https://www.youtube.com/watch?v=2HVKJg__wYk&t=...
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,https://www.youtube.com/watch?v=fomcMM1XA5A&t=...
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,https://www.youtube.com/watch?v=dMNIjzeZA1c&t=...
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U,https://www.youtube.com/watch?v=AwdMIo8Lv5U&t=42s
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
...,...,...,...,...,...,...
4101,can we do it again,243,244,can we do it again,GTQHOUT5T5E,https://www.youtube.com/watch?v=GTQHOUT5T5E&t=...
4102,sì va be my money,2154,2166,sì va be my money,j0ZS0wJ5XDk,https://www.youtube.com/watch?v=j0ZS0wJ5XDk&t=...
4103,so what did he do,103,106,so what did he do,1wxyoR21bo0,https://www.youtube.com/watch?v=1wxyoR21bo0&t=...
4104,we have y and t,31,34,we have y and t,h1Xh1JJuh_I,https://www.youtube.com/watch?v=h1Xh1JJuh_I&t=31s


In [264]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [265]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [266]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [267]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Italian_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'Italian_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [268]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [269]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass